In [1]:
!pip install pyensae

In [2]:
!pip install qgrid

In [3]:
%load_ext pyensae

In [5]:
%SQL_connect ./databases/baseball_full.db

In [6]:
%SQL_tables

['AllstarFull',
 'Appearances',
 'AwardsManagers',
 'AwardsPlayers',
 'AwardsShareManagers',
 'AwardsSharePlayers',
 'Batting',
 'BattingPost',
 'CollegePlaying',
 'Fielding',
 'FieldingOF',
 'FieldingOFsplit',
 'FieldingPost',
 'HallOfFame',
 'HomeGames',
 'Managers',
 'ManagersHalf',
 'Parks',
 'People',
 'Pitching',
 'PitchingPost',
 'Salaries',
 'Schools',
 'SeriesPost',
 'Teams',
 'TeamsFranchises',
 'TeamsHalf']

In [7]:
%SQL_schema Batting

{0: ('playerID', str),
 1: ('yearID', int),
 2: ('stint', int),
 3: ('teamID', str),
 4: ('lgID', str),
 5: ('G', int),
 6: ('AB', int),
 7: ('R', int),
 8: ('H', int),
 9: ('2B', int),
 10: ('3B', int),
 11: ('HR', int),
 12: ('RBI', float),
 13: ('SB', float),
 14: ('CS', float),
 15: ('BB', int),
 16: ('SO', float),
 17: ('IBB', float),
 18: ('HBP', float),
 19: ('SH', float),
 20: ('SF', float),
 21: ('GIDP', float),
 22: ('id', int)}

In [8]:
%%SQL
SELECT yearID, teamID, playerID, HR
FROM Batting
WHERE HR >= 50

,yearID,teamID,playerID,HR
0,1920,NYA,ruthba01,54
1,1921,NYA,ruthba01,59
2,1927,NYA,ruthba01,60
3,1928,NYA,ruthba01,54
4,1930,CHN,wilsoha01,56
5,1932,PHA,foxxji01,58
6,1938,BOS,foxxji01,50
7,1938,DET,greenha01,58
8,1947,PIT,kinerra01,51
9,1947,NY1,mizejo01,51


In [9]:
%%SQL
SELECT teamID, yearID, name, park
FROM Teams

,teamID,yearID,name,park
0,BS1,1871,Boston Red Stockings,South End Grounds I
1,CH1,1871,Chicago White Stockings,Union Base-Ball Grounds
2,CL1,1871,Cleveland Forest Citys,National Association Grounds
3,FW1,1871,Fort Wayne Kekiongas,Hamilton Field
4,NY2,1871,New York Mutuals,Union Grounds (Brooklyn)
5,PH1,1871,Philadelphia Athletics,Jefferson Street Grounds
6,RC1,1871,Rockford Forest Citys,Agricultural Society Fair Grounds
7,TRO,1871,Troy Haymakers,Haymakers' Grounds
8,WS3,1871,Washington Olympics,Olympics Grounds
9,BL1,1872,Baltimore Canaries,Newington Park


In [10]:
%%SQL
SELECT playerID, nameFirst, nameLast
FROM People

,playerID,nameFirst,nameLast
0,aardsda01,David,Aardsma
1,aaronha01,Hank,Aaron
2,aaronto01,Tommie,Aaron
3,aasedo01,Don,Aase
4,abadan01,Andy,Abad
5,abadfe01,Fernando,Abad
6,abadijo01,John,Abadie
7,abbated01,Ed,Abbaticchio
8,abbeybe01,Bert,Abbey
9,abbeych01,Charlie,Abbey


In [11]:
%%SQL
SELECT b.HR, t.name, t.park, p.nameFirst, p.nameLast
FROM
    (SELECT yearID, teamID, playerID, HR
    FROM Batting
    WHERE HR >= 50) AS b
JOIN
    (SELECT teamID, yearID, name, park
    FROM Teams
    ) AS t
ON b.teamID == t.teamID and b.yearID == t.yearID
JOIN
    (SELECT playerID, nameFirst, nameLast
    FROM People
    ) AS p
ON b.playerID == p.playerID

,HR,name,park,nameFirst,nameLast
0,54,New York Yankees,Polo Grounds IV,Babe,Ruth
1,59,New York Yankees,Polo Grounds IV,Babe,Ruth
2,60,New York Yankees,Yankee Stadium I,Babe,Ruth
3,54,New York Yankees,Yankee Stadium I,Babe,Ruth
4,56,Chicago Cubs,Wrigley Field,Hack,Wilson
5,58,Philadelphia Athletics,Shibe Park,Jimmie,Foxx
6,50,Boston Red Sox,Fenway Park II,Jimmie,Foxx
7,58,Detroit Tigers,Briggs Stadium,Hank,Greenberg
8,51,Pittsburgh Pirates,Forbes Field,Ralph,Kiner
9,51,New York Giants,Polo Grounds IV,Johnny,Mize


In [12]:
%%SQL --df=over50hr
SELECT yearID, teamID, playerID, HR
FROM Batting
WHERE HR >= 50

,yearID,teamID,playerID,HR
0,1920,NYA,ruthba01,54
1,1921,NYA,ruthba01,59
2,1927,NYA,ruthba01,60
3,1928,NYA,ruthba01,54
4,1930,CHN,wilsoha01,56
5,1932,PHA,foxxji01,58
6,1938,BOS,foxxji01,50
7,1938,DET,greenha01,58
8,1947,PIT,kinerra01,51
9,1947,NY1,mizejo01,51


In [13]:
over50hr.head()

,yearID,teamID,playerID,HR
0,1920,NYA,ruthba01,54
1,1921,NYA,ruthba01,59
2,1927,NYA,ruthba01,60
3,1928,NYA,ruthba01,54
4,1930,CHN,wilsoha01,56


## R dplyr magic

In [14]:
!pip install rpy2

    100% |████████████████████████████████| 122kB 5.1MB/s ta 0:00:01
  Stored in directory: /Users/bn8210wy/Library/Caches/pip/wheels/a8/70/ed/8ed99930ef5a7ba56f7bacb08d9363ecd29bc289eb80c41554
Successfully built rpy2


In [16]:
!pip install tzlocal

  Stored in directory: /Users/bn8210wy/Library/Caches/pip/wheels/15/ae/df/a67bf1ed84e9bf230187d36d8dcfd30072bea0236cb059ed91
Successfully built tzlocal


In [17]:
%load_ext rpy2.ipython

In [20]:
%%R
library(dplyr)

In [21]:
%%R
batting <- read.csv('./data/baseball/core/Batting.csv')
batting %>% head

   playerID yearID stint teamID lgID  G  AB  R  H X2B X3B HR RBI SB CS BB SO
1 abercda01   1871     1    TRO <NA>  1   4  0  0   0   0  0   0  0  0  0  0
2  addybo01   1871     1    RC1 <NA> 25 118 30 32   6   0  0  13  8  1  4  0
3 allisar01   1871     1    CL1 <NA> 29 137 28 40   4   5  0  19  3  1  2  5
4 allisdo01   1871     1    WS3 <NA> 27 133 28 44  10   2  2  27  1  1  0  2
5 ansonca01   1871     1    RC1 <NA> 25 120 29 39  11   3  0  16  6  2  2  1
6 armstbo01   1871     1    FW1 <NA> 12  49  9 11   2   1  0   5  0  1  0  1
  IBB HBP SH SF GIDP
1  NA  NA NA NA    0
2  NA  NA NA NA    0
3  NA  NA NA NA    1
4  NA  NA NA NA    0
5  NA  NA NA NA    0
6  NA  NA NA NA    0


In [22]:
%%R
(batting
 %>% select(playerID, yearID, teamID, HR)
 %>% filter(HR >= 50)
 %>% head
)

   playerID yearID teamID HR
1  ruthba01   1920    NYA 54
2  ruthba01   1921    NYA 59
3  ruthba01   1927    NYA 60
4  ruthba01   1928    NYA 54
5 wilsoha01   1930    CHN 56
6  foxxji01   1932    PHA 58


In [24]:
%%R
(batting
 %>% select(playerID, yearID, teamID, HR)
 %>% filter(HR >= 50)
) -> over50hr
over50hr %>% head

   playerID yearID teamID HR
1  ruthba01   1920    NYA 54
2  ruthba01   1921    NYA 59
3  ruthba01   1927    NYA 60
4  ruthba01   1928    NYA 54
5 wilsoha01   1930    CHN 56
6  foxxji01   1932    PHA 58
